## 상권 분석 ##

카카오 api를 활용하여 점포의 상권을 분석할 수 있다. 사업자등록증이 있는 상업시설이 나오며, 주택가 등의 자료는 나오지 않아 추가 확인이 필요하다
또한 반경 거리를 설정하여 점포를 중심으로 반경 내에 위치한 점포를 추출할 수 있다 

상권의 기준을 선정함이 우선되어야한다. 점포별 주변 상업시설은 추출 가능하지만 상권 기준을 만들어 관리 방안을 수립하는 것이 우선되어야 한다. 
이후 점별 상업시설을 추출하여 점별 상권을 정렵한다   * 8대입지 or 15대입지를 표준으로 사용하며 / 닐슨의 상권 정립 원칙을 참고함이 좋다

In [ ]:
import requests
import csv
from tqdm import tqdm
from collections import OrderedDict

In [40]:
def get_places(location, radius, category_group_code):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {
        "Authorization": "개인 카카오 api"
    }
    params = {
        "category_group_code": category_group_code,
        "x": location[0],
        "y": location[1],
        "radius": radius,
        "sort": "distance",
        "page": 1,
        "size": 15
    }

    places = []

    while True:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        for place in data['documents']:
            places.append(place)

        meta = data['meta']
        is_end = meta['is_end']
        if is_end:
            break

        params['page'] += 1

    return places


# ... (기타 코드는 변경되지 않음) ...

def main():
    location_csv = '점포 주소.csv'
    radius = 150
    category_group_codes = ['MT1', 'CS2', 'SC4', 'BK9', 'PS3', 'AC5', 'PK6', 'OL7', 'SW8', 'CT1', 'AG2', 'PO3', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9']

    # 아마도 이러한 함수는 다른 곳에서 정의되어 있다고 가정합니다.
    addresses = get_location_from_csv(location_csv)
    store_names = get_store_names_from_csv(location_csv)

    places = []  # places 리스트를 초기화

    new_store_names = []  # 새로운 상점 이름을 저장할 별도의 리스트 생성

    # tqdm 사용하여 진행 상황 표시
    for address, store_name in tqdm(list(zip(addresses, store_names)), desc="장소 추출 중", unit="주소"):
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {
            "Authorization": "개인 카카오 api"
        }
        params = {
            "query": address
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'documents' in data and len(data['documents']) > 0:
            coordinates = data['documents'][0]['address']
            location = [float(coordinates['x']), float(coordinates['y'])]

            for category_group_code in category_group_codes:
                places.extend(get_places(location, radius, category_group_code))
                new_store_names.extend([store_name] * len(get_places(location, radius, category_group_code)))

    store_names.extend(new_store_names)  # 루프 후에 store_names 리스트를 확장

    matched_places = match_address_name(addresses, places)
    save_to_csv(matched_places, '상업시설.csv', store_names)

# ... (기타 코드는 변경되지 않음) ...

def get_store_names_from_csv(filename):
    store_names = []

    with open(filename, 'r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row

        for row in reader:
            store_name = row[1]
            store_names.append(store_name)

    return store_names

if __name__ == '__main__':
    main()


장소 추출 중: 100%|█████████████████████████| 117/117 [14:43<00:00,  7.55s/주소]
